In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None)
from pandas.plotting import register_matplotlib_converters; register_matplotlib_converters()
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
import statistics as stat
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from pandas.plotting import register_matplotlib_converters
from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, wtf
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.UPMhelper import UPMalerts

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [11]:
query = """
        SELECT id as account_id, account_group_id
        FROM scrubbed_exchange_accounts
        LIMIT 40
        """
query = """
        SELECT min(id) as account_id, min(account_group_id)
        FROM scrubbed_exchange_accounts
        """
query = """
        SELECT count(distinct id) as account_id, count(distinct account_group_id), 1.0*count(distinct id)/count(distinct account_group_id) as avg
        FROM scrubbed_exchange_accounts
        """
data0 = execute_query_data_frame(query, 'gemini', ssh = 'datalab_prod')
data0

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.648463


INFO:root:Loaded dataframe with 1 rows in 0:00:02.031394.


,account_id,count,avg
0,1226196,1226039,1.000128


#### User/Group Restrictions for Account_ID

In [39]:
query = """
        SELECT * from user_account_roles
        """

data0 = execute_query_data_frame(query, 'gemini', ssh = 'datalab_prod')
data0.sort_values('exchange_account_id')

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.698189


INFO:root:Loaded dataframe with 24 rows in 0:00:01.017482.


,exchange_user_id,exchange_account_id,role,last_modified_at,attestation_user_id_account_id
6,906131,874521,Administrator,2019-07-18 20:53:44.390227,"[b',', b'\x1f', b'?', b'n', b'I', b'\x98', b'l..."
2,911859,907903,Administrator,2019-07-18 20:53:05.313686,"[b'\x94', b'\x92', b'\xc1', b'\xf3', b'\xb3', ..."
3,1085007,907903,Administrator,2019-07-18 20:53:05.313686,"[b'=', b'\x9e', b'i', b'\xe9', b'\x1b', b'\xeb..."
11,958241,938926,Trader,2019-07-18 20:54:23.709771,"[b'j', b'\xff', b'\xa4', b'b', b'\xb4', b'\x89..."
10,1048575,938926,Administrator,2019-07-18 20:54:23.709771,"[b'%', b'S', b'4', b't', b'\xf5', b'\xf9', b'R..."
8,958595,938926,Trader,2019-07-18 20:54:23.709771,"[b'%', b'\x98', b'\xa2', b'J', b'~', b'p', b'\..."
7,958602,938926,Trader,2019-07-18 20:54:23.709771,"[b'\x19', b' ', b'L', b'X', b'\x8a', b'\xab', ..."
9,958018,938926,Administrator,2019-07-18 20:54:23.709771,"[b'i', b'\xe1', b'9', b'\xf0', b'\xf9', b'\xbe..."
12,958602,1355262,Auditor,2019-07-18 20:54:23.709771,"[b'\x06', b'l', b'\xac', b'f', b'\xd2', b'\xfd..."
13,958595,1355262,Auditor,2019-07-18 20:54:23.709771,"[b'K', b'\x85', b'\xa7', b'm', b'(', b'\xd8', ..."


In [37]:
data0.groupby(['exchange_account_id','exchange_user_id']).count()

role  last_modified_at  \
exchange_account_id exchange_user_id                           
874521              906131               1                 1   
907903              911859               1                 1   
                    1085007              1                 1   
938926              958018               1                 1   
                    958241               1                 1   
                    958595               1                 1   
                    958602               1                 1   
                    1048575              1                 1   
1355262             958018               1                 1   
                    958241               1                 1   
                    958595               1                 1   
                    958602               1                 1   
                    1048575              1                 1   
1355268             958018               1                 1   
                    958241               1                 1   
                    958595               1                 1   
                    958602               1                 1   
                    1048575              1                 1   
1355283             911859               1                 1   
                    1085007              1                 1   
1355301             305633               1                 1   
                    319549               1                 1   
1355302             925209               1                 1   
                    925266               1                 1   

                                      attestation_user_id_account_id  
exchange_account_id exchange_user_id                                  
874521              906131                                         1  
907903              911859                                         1  
                    1085007                                        1  
938926              958018                                         1  
                    958241                                         1  
                    958595                                         1  
                    958602                                         1  
                    1048575                                        1  
1355262             958018                                         1  
                    958241                                         1  
                    958595                                         1  
                    958602                                         1  
                    1048575                                        1  
1355268             958018                                         1  
                    958241                                         1  
                    958595                                         1  
                    958602                                         1  
                    1048575                                        1  
1355283             911859                                         1  
                    1085007                                        1  
1355301             305633                                         1  
                    319549                                         1  
1355302             925209                                         1  
                    925266                                         1

In [35]:
pd.unique(data0['exchange_account_id'])

array([1355301,  907903, 1355283,  874521,  938926, 1355262, 1355268,
       1355302])

In [18]:
query = """
        SELECT 
        uagr.exchange_user_id as user_id, uagr.account_group_id as group_id, 
        ea.id as account_id, 
        ag.restriction as group_restriction,
        eu.restriction as user_restriction,
        ua.exchange_account_id as account_idUA,
        ua2.exchange_user_id as user_idUA
        
        FROM user_account_group_roles AS uagr
        
        LEFT join exchange_accounts as ea using(account_group_id)
        LEFT join account_group as ag using(account_group_id)      
        LEFT join exchange_users as eu on uagr.exchange_user_id = eu.id
        LEFT join user_account_roles as ua using(exchange_user_id)
        LEFT join user_account_roles as ua2 on ea.id = ua.exchange_account_id
        
        """
#        where ag.restriction = 'Closed' or eu.restriction = 'Closed'

data0 = execute_query_data_frame(query, 'gemini', ssh = 'datalab_prod')
data0.isna().sum(axis=0)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.704016


INFO:root:Loaded dataframe with 1142178 rows in 0:01:23.347214.


user_id                    0
group_id                   0
account_id                 0
group_restriction    1140458
user_restriction      888967
account_idua         1142078
user_idua            1142082
dtype: int64

NameError: name 'execute_query_data_frame' is not defined

In [28]:
test = data0.loc[(~data0['user_idua'].isna()) | (~data0['account_idua'].isna())]
len(pd.unique(test['account_idua']))

2

In [53]:
data0['restriction'] = pd.DataFrame(np.where(data0['group_restriction']=='Closed', 'group','user'))
data1 = data0.set_index('account_id')
data1.isna().sum(axis=0)

NameError: name 'none' is not defined

#### Read in Blockchain Report

In [31]:
report0 = pd.read_excel('~/Downloads/Wall Street Market.xlsx', sheet_name = 'Sheet1').reset_index(drop=True).rename(columns={'Unnamed: 1':'account_id'})
report1 = report0.set_index('account_id')
report1.shape

(397, 1)

#### Combined

In [43]:
combined0 = report1.merge(data1, on='account_id', how='left').drop(columns=['UserID','group_restriction','user_restriction']).sort_index()
combined0.shape

(397, 3)

In [44]:
combined0.head()

,user_id,group_id,restriction
account_id,,,
1988,NaN,NaN,NaN
2117,NaN,NaN,NaN
5340,NaN,NaN,NaN
10079,NaN,NaN,NaN
10094,NaN,NaN,NaN


In [50]:
combined0['restriction'].value_counts()

group    52
user     17
Name: restriction, dtype: int64

In [47]:
combined0.to_csv('~/Desktop/Closed.csv')